Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from random import random
from math import ceil
from functools import reduce
from collections import namedtuple, deque
from queue import PriorityQueue

import numpy as np
from tqdm.auto import tqdm

c:\Users\darda\AppData\Local\pypoetry\Cache\virtualenvs\prova-sVNX-V2m-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
State = namedtuple('State', ['taken', 'not_taken'])

def join_taken(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

def goal_check(state):
    return np.all(join_taken(state))

In [3]:
PROBLEM_SIZE = 10
NUM_SETS = 10
SETS = list(np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))

assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"


In [4]:
# preproccesing


In [5]:

def missing_coverage(state):
    return PROBLEM_SIZE-sum(state)

def calc_overlap_and_false_number(state,e):
    return sum( np.logical_and(
        state, SETS[e])) + missing_coverage(SETS[e])

def calc_extension(state,e):
    ext=0
    for i in state.taken:
        for j in range(PROBLEM_SIZE):
            if SETS[i][j]==False and SETS[e][j]==True:
                ext+=1
    return ext

def my_h(state,e):    
    return calc_overlap(state, e)+calc_extension(state,e)

def my_h2(state):
    joined_taken=join_taken(state)
    
    not_taken_sorted_by_overlap=sorted(state.not_taken,key=lambda e:calc_overlap_and_false_number(joined_taken,e))    

    sets_still_needed=0
    for s in not_taken_sorted_by_overlap:
        
        if missing_coverage(joined_taken)==0:
            break
        joined_taken = np.logical_or(joined_taken,SETS[s])
        sets_still_needed+=1
        
    return sets_still_needed    

def f(state):
    # return len(state.taken)+my_h(state,e)
    return len(state.taken) + my_h2(state)

In [6]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((0, state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state.not_taken:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f(new_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

9it [00:00, 474.93it/s]

Solved in 9 steps (4 tiles)


In [7]:
current_state.taken

{3, 4, 5, 8}